In [1]:
from sub_func import *
from pf_selection import *
from pipeline_utils import *

from t1 import *
from t2_t4 import *
from t5 import *

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


예시로, 2023년 1분기 분석을 위해 2022년 4분기 정보를 바탕으로 2023년 1분기 초기 포트폴리오 생성

In [2]:
# 조회할 정보의 기간
base_year = '2023'
base_quarter = 'Q2'
start_date = '20230331'
end_date = '20230630'

# 포트폴리오를 활용하는 실제 기간 (ex. 2022 Q4를 조회하여 2023 Q1을 위한 포트폴리오 생성)
target_year = '2023'
target_quarter = 'Q3'

# 오늘 날짜 (코스피 장이 열린 날만 한정, 예시)
kospi_open_days = get_kospi_open_days("20230701", "20230930")

# 'pf_selection_agent'에 초기 포트폴리오가 있는지 확인
db_info = get_all_page_ids_from_database('pf_selection_agent')
init_pf_page_title = f"{target_year}_{target_quarter}_init_pf"

if init_pf_page_title not in list(db_info.keys()):
    print('초기 포트폴리오를 생성합니다...')
    pf_selection_info = pf_selection_main(base_year, base_quarter, start_date, end_date)
    print("노션에 output을 모두 저장합니다...")
    to_DB('pf_selection_agent', init_pf_page_title, f"{start_date}_{end_date}", str(pf_selection_info))

# 'pf_selection_agent'에서 종목 코드 가져오기
temp_tickers = get_tickers_from_json_v2('pf_selection_agent', f'{target_year}_{target_quarter}_init_pf')
tickers = []

for ticker in temp_tickers:
    if not is_delisted(ticker, today):
        tickers.append(ticker)
        
weight_dict = {ticker: 1 / len(tickers) for ticker in tickers}
 
# JSON 파일 저장할 디렉토리 경로
log_dir = f"./pf_logs/{target_year}_{target_quarter}"
os.makedirs(log_dir, exist_ok=True)  # 디렉토리 없으면 생성

# 파일 경로
file_path = os.path.join(log_dir, f"{target_year}_{target_quarter}_init_pf.json")

with open(f'{file_path}', "w", encoding="utf-8") as f:
    json.dump(weight_dict, f, indent=2, ensure_ascii=False)

In [3]:
tickers = temp_tickers

tickers

['042700',
 '195870',
 '077500',
 '000990',
 '029460',
 '092220',
 '007810',
 '011420',
 '036570',
 '180640']

In [5]:
def finTF(today, tickers, base_year, base_quarter, target_year, target_quarter, start_date):

    print('***** t1_analyst_main 실행 *****')
    t1_analyst_main(today, tickers, base_year, base_quarter)

    print('***** t1_pf_manager_main 실행 *****')
    t1_pf_manager_main(today, tickers, base_year, base_quarter, target_year, target_quarter)

    print('***** t1_trader_main 실행 *****')
    t1_trader_main(today, tickers, base_year, base_quarter)

    print('***** t2_t4_main 실행 *****')
    t_2_t_4_main(tickers, start_date, today, target_year, target_quarter, base_year, base_quarter)

    print('***** t5_main 실행 *****')
    t_5_main(today, base_year, base_quarter)

In [7]:
kospi_open_days[57]

'20230921'

In [8]:
test_span = kospi_open_days[58:]
# 0719부터 실행 

for today in test_span:
    finTF(today, tickers, base_year, base_quarter, target_year, target_quarter, start_date)
    print(f'{today}일자 실행 완료')

***** t1_analyst_main 실행 *****
[t1_analyst] === 종목별 분석 시작 ===
UK에 대해 Inflation Rate 정보를 찾을 수 없습니다. | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/FRED/UK/Inflation Rate/2023/2023_Inflation Rate.csv'
JP에 대해 Inflation Rate 정보를 찾을 수 없습니다. | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/FRED/JP/Inflation Rate/2023/2023_Inflation Rate.csv'

=== 042700 분석 중... ===
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-AyK9q0CbRlX9uI4mCjhASxTjQBIOW', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무 분석 보고서\n\n## 1. 재무제표 요약\n### 1.1 연결재무상태표\n- **유동자산**: 258,479억원 (전기: 266,177억원) - 감소\n- **현금 및 현금성 자산**: 91,584억원 (전기: 90,896억원) - 증가\n- **매출채권**: 58,549억원 (전기: 80,474억원) - 감소\n- **재고자산**: 77,209억원 (전기: 80,082억원) - 감소\n- **비유동자산**: 363,646억원 (전기: 185,868억원) - 증가\n- **총자산**: 622,125억원 (전기: 455,433억원) - 증가\n

/Users/yeonsuk/investment/finTF/pipeline/t1.py:470: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 195870 분석 중... ===
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-AyKBal6grA5yQ1jiCdYr20EA3yAWJ', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무 분석 보고서\n\n## 1. 재무제표 요약\n\n### 1.1 재무상태표\n- **유동자산**: 357,062,612천원 (전기: 371,321,313천원) - 감소\n- **비유동자산**: 307,471,641천원 (전기: 288,292,509천원) - 증가\n- **자산총계**: 664,534,253천원 (전기: 659,613,822천원) - 증가\n- **유동부채**: 152,555,112천원 (전기: 196,470,095천원) - 감소\n- **비유동부채**: 22,886,974천원 (전기: 15,848,433천원) - 증가\n- **부채총계**: 175,442,085천원 (전기: 212,318,528천원) - 감소\n- **자본총계**: 489,092,168천원 (전기: 447,295,294천원) - 증가\n\n### 1.2 손익계산서\n- **매출액**: 371,617,020천원 \n- **영업이익**: 66,819,114천원 \n- **당기순이익**: 56,674,342천원 \n- **주당순이익**: 3,334원/주\n\n### 1.3 주요 재무 비율\n- **주가**: 60,600원\n- **PER**: 0.000035\n- **PBR**: 0.0000021\n- **ROE**: 6.01%\n- **Profit Growth Rate**: -35.81%\n- **CAGR**: -0.36%\n\n## 2. 분석 및 해석\n\n### 2.1 자산과 부채\n- **자산 증가**: 총 자산이 증가했으나 유동자산이 감소하면서 비유동자산의 비중이 증가했습니다. 이는 기업이 장기적 투자에 집중하고 있

/Users/yeonsuk/investment/finTF/pipeline/t1.py:470: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 077500 분석 중... ===
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/077500/_077500_재무제표 ().csv'
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/077500/_077500_재무제표 ().csv'
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-AyKDChLI4Z7mU3NS0VkP7gqkzUPiz', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무 분석 보고서\n\n## 1. 요약재무정보 분석\n\n### 1.1 연결재무상태표 요약\n\n- **유동자산**은 388,670백만원으로, 전기(397,108백만원) 대비 감소하였으나, 전년 동기 대비(271,743백만원)로는 크게 증가하였습니다. 이는 회사의 유동성이 향상되었음을 나타냅니다.\n- **비유동자산**은 217,769백만원으로, 소폭 증가하였습니다. 투자자산의 증가가 주효했음을 알 수 있습니다.\n- **부채총계**는 245,494백만원으로, 전기(262,882백만원) 대비 감소하였으나, 유동부채의 감소가 주효했습니다.\n- **자본총계**는 360,945백만원으로, 전기(347,841백만원) 대비 증가하였습니다. 이는 자본구성 요소인 이익

...

[DEBUG] 외국인 보유 비중 데이터 로드 성공
[DEBUG] 데이터 병합 완료
[DEBUG] 최종 데이터 shape: (62, 5)
[DEBUG] 학습 데이터 통계:
X_train shape: (47, 15, 3)
y_train shape: (47,)
X_train 값 범위: 0.0 ~ 1.0
y_train 값 범위: 0.0 ~ 0.9905213270142181
Epoch 1/50


/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.3041 - mae: 0.7146 - val_loss: 0.0014 - val_mae: 0.0450 - learning_rate: 0.0010
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1860 - mae: 0.5479 - val_loss: 0.0030 - val_mae: 0.0727 - learning_rate: 0.0010
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1255 - mae: 0.4359 - val_loss: 0.0062 - val_mae: 0.1040 - learning_rate: 0.0010
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0790 - mae: 0.3402 - val_loss: 0.0113 - val_mae: 0.1458 - learning_rate: 0.0010
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0402 - mae: 0.2385 - val_loss: 0.0187 - val_mae: 0.1899 - learning_rate: 0.0010
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0289 - mae: 0.2153 - val_loss: 0.0277 - val_mae: 0.2327 - learning_rate: 0.0010
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0214 - mae: 0.1837 - val_loss: 0.0318 - val_mae: 0.2498 - learning_rate: 5.0000e-04
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - loss: 0.1802 - mae: 0.5504 - val_loss: 0.2499 - val_mae: 0.7015 - learning_rate: 0.0010
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1435 - mae: 0.4718 - val_loss: 0.2102 - val_mae: 0.6427 - learning_rate: 0.0010
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1061 - mae: 0.3901 - val_loss: 0.1733 - val_mae: 0.5825 - learning_rate: 0.0010
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0824 - mae: 0.3226 - val_loss: 0.1410 - val_mae: 0.5241 - learning_rate: 0.0010
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0528 - mae: 0.2288 - val_loss: 0.1124 - val_mae: 0.4662 - learning_rate: 0.0010
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0371 - mae: 0.1966 - val_loss: 0.0869 - val_mae: 0.4077 - learning_rate: 0.0010
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0289 - mae: 0.1962 - val_loss: 0.0656 - val_mae: 0.3512 - learning_rate: 0.0010
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/

In [18]:
kospi_open_days[10]

'20230116'